### Chatbot Assistant with Persistent Memory and Tool Use using LangChain v1.0+, Ollama, and Gradio v6.1.0

This implementation showcases a fully local, tool-enabled conversational assistant built with LangChain v1.0+, powered by open-source large language models served via Ollama, and delivered through a Gradio v6.1.0 user interface. The architecture is designed for robustness, extensibility, and compatibility with non-OpenAI models while preserving conversational memory across turns.

---

## Key Features and Design Updates

### Local LLM Integration (Ollama)
- Uses `langchain_ollama.ChatOllama` instead of `ChatOpenAI`
- Runs entirely on local infrastructure via an Ollama server (`base_url="http://localhost:11434"`)
- Deterministic responses enabled by setting `temperature=0`
- Model selection is configurable (default: `qwen2.5:3b`)

### Agent-Based Tool Usage
- Leverages LangChain’s `create_agent` abstraction
- Accepts a custom `system_prompt` at agent creation time
- Enables tool calling for open-source models that do not support OpenAI function calling
- Supports arbitrary tools such as:
  - Wikipedia search
  - Weather lookups
  - Custom user-defined tools

### Persistent Conversation Memory
- Uses `ChatMessageHistory` for structured and reliable memory management
- Maintains a complete message history across turns
- Injects full conversation context into each agent invocation
- Allows the assistant to recall, reference, and reason over prior interactions

### Explicit Message Construction
- Manually constructs the message list using LangChain message objects
- Replays stored messages from `ChatMessageHistory` on each request
- Appends the current `HumanMessage` before invoking the agent
- Extracts the final assistant response from the returned message list

### Error Handling and Stability
- Wraps agent invocation in a `try/except` block to handle local LLM failures
- Logs full Python tracebacks for debugging and observability
- Returns a graceful user-facing error message when exceptions occur

### Gradio v6.1.0 User Interface
- Built with `gr.Blocks` for layout flexibility
- Includes:
  - Scrollable chat window
  - Multi-line text input
  - Send and clear-history controls
  - Example prompts for quick interaction
- Displays live metadata about memory usage (message count)

### UI and Memory State Separation
- Internal conversation state is managed exclusively by LangChain (`ChatMessageHistory`)
- Gradio chat state is maintained independently for rendering
- Ensures clean separation between UI concerns and agent logic
- Clear-history action resets both UI state and internal memory

### Modular and Extensible Architecture
- Tools are injected at initialization time
- Model configuration is parameterized
- UI creation, memory management, and agent execution are clearly separated
- Suitable as a foundation for more advanced local AI assistants

---

## Summary

This project provides a production-ready reference architecture for building local, tool-augmented conversational agents with persistent memory using LangChain’s modern agent APIs. It avoids reliance on proprietary APIs or OpenAI-specific features, making it well suited for open-source LLM deployments, privacy-sensitive environments, and offline-first applications.

*Based on the final project from the course **“Functions, Tools and Agents with LangChain”** by deeplearning(dot)ai* fully updated to LangChain v1+, with a new custom UI based on Gradio and a local LLM running via Ollama.

In [ ]:
import gradio as gr
import param
from langchain_ollama import ChatOllama
from langchain.agents import create_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_community.chat_message_histories import ChatMessageHistory

In [ ]:
class chatbot_demo(param.Parameterized):
    
    def __init__(self, tools, model_name="qwen2.5:3b", **params):
        super(chatbot_demo, self).__init__(**params)
        self.tools = tools
        
        # Use LangChain's ChatMessageHistory for better memory management
        self.chat_history = ChatMessageHistory()
        
        # System message
        self.system_prompt = "You are a helpful, smart and jovial assistant who remembers the conversation history"
        
        # Create llm with Ollama
        self.model = ChatOllama(
            model=model_name,
            temperature=0.0, # increase for less deterministic results
            base_url="http://localhost:11434"
        )
        
        # Create the agent using create_agent with system_prompt parameter
        self.agent = create_agent(
            model=self.model,
            tools=self.tools,
            system_prompt=self.system_prompt
        )
    
    def process_chain(self, message, history):
        """
        Process user message and return response with memory retention.
        
        Args:
            message: Current user message
            history: List of [user_msg, bot_msg] pairs from Gradio
        
        Returns:
            Bot response string
        """
        if not message:
            return ""
        
        try:
            # Build messages list including full history
            messages = []
            
            # Get all messages from internal chat history
            for msg in self.chat_history.messages:
                messages.append(msg)
            
            # Add current user message
            messages.append(HumanMessage(content=message))
            
            # Invoke agent with full message history
            result = self.agent.invoke({"messages": messages})
            
            # Extract bot response (last message in the result)
            bot_response = result["messages"][-1].content
            
            # Store in chat history for future turns
            self.chat_history.add_user_message(message)
            self.chat_history.add_ai_message(bot_response)
            
            return bot_response
            
        except Exception as e:
            import traceback
            error_details = traceback.format_exc()
            print(f"Error details:\n{error_details}")
            return f"Sorry, I encountered an error: {str(e)}"
    
    def clear_history(self):
        """Clear conversation memory"""
        self.chat_history.clear()
        return None
    
    def get_history_summary(self):
        """Get a summary of conversation history"""
        return f"Total messages in history: {len(self.chat_history.messages)}"
    
    def create_ui(self):
        """Create and return Gradio ChatInterface"""
        
        with gr.Blocks() as demo:
            gr.Markdown("# Smart and Jovial Assistant Chatbot")
            gr.Markdown("Ask me anything! I'll remember our conversation.")
            
            chatbot = gr.Chatbot(height=500)

            msg = gr.Textbox(
                placeholder="Type your message here...",
                label="Message",
                lines=2
            )
            
            with gr.Row():
                submit = gr.Button("Send", variant="primary")
                clear = gr.Button("Clear History")
            
            # History info display
            history_info = gr.Markdown("Messages in memory: 0")
            
            # Examples
            gr.Examples(
                examples=[
                    "What can you help me with?",
                    "Search Wikipedia for Deep Learning",
                    "What's the temperature in Paris? (latitude: 48.857817, longitude: 2.295198)",
                    "What did we talk about earlier?"
                ],
                inputs=msg
            )
            
            def respond(message, chat_history):
                if not message:
                    return "", chat_history, self.get_history_summary()
            
                if chat_history is None:
                    chat_history = []
            
                bot_message = self.process_chain(message, chat_history)
                
                # Format for Gradio's "messages" type
                chat_history = chat_history + [
                    {"role": "user", "content": message},
                    {"role": "assistant", "content": bot_message}
                ]
            
                return "", chat_history, self.get_history_summary()
            
            
            def clear_all():
                self.clear_history()
                return [], "Messages in memory: 0"

            
            # Event handlers
            submit.click(respond, [msg, chatbot], [msg, chatbot, history_info])
            msg.submit(respond, [msg, chatbot], [msg, chatbot, history_info])
            clear.click(clear_all, None, [chatbot, history_info])
        
        return demo

### Tools available to the chatbot

In [ ]:
from langchain.tools import tool
import wikipedia
import requests
import datetime
from pydantic import BaseModel, Field
from typing import Optional

In [ ]:
# Create your tool
@tool
def create_your_tool(query: str) -> str:
    """This function can do whatever you would like once you fill it in """
    print(type(query))
    return query[::-1]

In [ ]:
# Wikipedia tool
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[:3]:
        try:
            wiki_page = wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            wikipedia.exceptions.PageError,
            wikipedia.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [ ]:
import datetime
import requests
from typing import Optional
from pydantic import BaseModel, Field, field_validator
from langchain.tools import tool


class OpenMeteoInput(BaseModel):
    city: Optional[str] = Field(
        None, 
        description="City or location name (e.g., 'Paris', 'Nevada', 'Cracow, 'Fuerteventura')"
    )
    country: Optional[str] = Field(
        None, 
        description="Country name or ISO code (e.g., 'Spain', 'USA', 'Poland', 'ES'). Optional but improves accuracy."
    )
    latitude: Optional[float] = Field(
        None, 
        description="Latitude coordinate. Only use if explicitly provided by user."
    )
    longitude: Optional[float] = Field(
        None, 
        description="Longitude coordinate. Only use if explicitly provided by user."
    )

@tool(args_schema=OpenMeteoInput)
def get_weather_conditions(
    city: Optional[str] = None,
    country: Optional[str] = None,
    latitude: Optional[float] = None,
    longitude: Optional[float] = None
) -> str:
    """
    Get current weather conditions (temperature, precipitation, wind, humidity) for any location.
    Use this whenever you need weather information to answer a question, whether explicitly asked or contextually needed.
    Automatically geocodes city names to coordinates.
    """
    
    # Geocode if coordinates not provided
    if latitude is None or longitude is None:
        if not city:
            return "Error: Please provide either a location name or both latitude and longitude."
        
        location_query = f"{city}, {country}" if country else city
        
        geocode_url = "https://nominatim.openstreetmap.org/search"
        geocode_params = {
            'q': location_query,
            'format': 'json',
            'limit': 1
        }
        headers = {'User-Agent': 'LangChain-Weather-Bot/1.0'}
        
        try:
            geo_response = requests.get(geocode_url, params=geocode_params, headers=headers, timeout=10)
            geo_response.raise_for_status()
            geo_data = geo_response.json()
            
            if not geo_data:
                return f"Error: Could not find coordinates for '{location_query}'."
            
            latitude = float(geo_data[0]['lat'])
            longitude = float(geo_data[0]['lon'])
            location_name = geo_data[0].get('display_name', location_query)
            
        except Exception as e:
            return f"Error geocoding location: {str(e)}"
    else:
        location_name = f"coordinates ({latitude}, {longitude})"
    
    # Fetch weather data
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'current': 'temperature_2m,relative_humidity_2m,precipitation,rain,weather_code,wind_speed_10m,wind_direction_10m',
        'forecast_days': 1,
        'temperature_unit': 'celsius',
        'wind_speed_unit': 'kmh',
    }
    
    try:
        response = requests.get(BASE_URL, params=params, timeout=10)
        response.raise_for_status()
        results = response.json()
    except Exception as e:
        return f"Error fetching weather data: {str(e)}"
    
    # Parse current weather
    current = results.get('current', {})
    temp = current.get('temperature_2m', 'N/A')
    humidity = current.get('relative_humidity_2m', 'N/A')
    precipitation = current.get('precipitation', 0)
    rain = current.get('rain', 0)
    wind_speed = current.get('wind_speed_10m', 'N/A')
    wind_direction = current.get('wind_direction_10m', 'N/A')
    weather_code = current.get('weather_code', 0)
    
    # Decode weather condition
    weather_descriptions = {
        0: "Clear sky", 1: "Mainly clear", 2: "Partly cloudy", 3: "Overcast",
        45: "Foggy", 48: "Depositing rime fog",
        51: "Light drizzle", 53: "Moderate drizzle", 55: "Dense drizzle",
        61: "Slight rain", 63: "Moderate rain", 65: "Heavy rain",
        71: "Slight snow", 73: "Moderate snow", 75: "Heavy snow", 77: "Snow grains",
        80: "Slight rain showers", 81: "Moderate rain showers", 82: "Violent rain showers",
        85: "Slight snow showers", 86: "Heavy snow showers",
        95: "Thunderstorm", 96: "Thunderstorm with slight hail", 99: "Thunderstorm with heavy hail"
    }
    weather_condition = weather_descriptions.get(weather_code, "Unknown")
    
    # Build report
    report = (
        f"Weather in {location_name}:\n"
        f"Temperature: {temp}°C\n"
        f"Conditions: {weather_condition}\n"
        f"Humidity: {humidity}%\n"
        f"Wind: {wind_speed} km/h from {wind_direction}°\n"
        f"Precipitation: {precipitation} mm (rain: {rain} mm)"
    )
    
    return report

### User Interface (Gradio)

In [ ]:
# Run the Chatbot UI 
if __name__ == "__main__":
    
    # Create agent with tools
    tools = [create_your_tool,search_wikipedia, get_weather_conditions]
    chatbot = chatbot_demo(tools=tools, model_name="qwen2.5:3b")
    
    # Launch interface
    interface = chatbot.create_ui()
    interface.launch(share=False, debug=True)